# Day 15 - Chiton

https://adventofcode.com/2021/day/15

In [1]:
from pathlib import Path

INPUTS = Path("input.txt").read_text().strip().split("\n")
MAP = [list(map(int, x)) for x in INPUTS]


## Part 1

Clearly this is a path-finding challenge, and I don't do that shtuff (sic).

I at least know this is something that I *think* [Dijkstra's algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) can solve. So, I'm going to try implementing that, based on details from [this source](https://pythonwife.com/dijkstras-algorithm-in-python/) (with refactors for proper Python code style).

In [10]:
from collections import defaultdict


class Graph:
    def __init__(self):
        self.nodes = set()
        self.edges = defaultdict(list)
        self.distances = {}

    def add_node(self, value):
        self.nodes.add(value)

    def add_edge(self, from_node, to_node, distance):
        self.edges[from_node].append(to_node)
        self.distances[(from_node, to_node)] = distance


def dijkstra(graph, initial):
    visited = {initial: 0}
    path = defaultdict(list)

    nodes = set(graph.nodes)

    while nodes:
        min_node = None
        for node in nodes:
            if node in visited:
                if min_node is None:
                    min_node = node
                elif visited[node] < visited[min_node]:
                    min_node = node
        if min_node is None:
            break

        nodes.remove(min_node)
        current_weight = visited[min_node]

        for edge in graph.edges[min_node]:
            weight = current_weight + graph.distances[(min_node, edge)]
            if edge not in visited or weight < visited[edge]:
                visited[edge] = weight
                path[edge].append(min_node)

    return visited, path


def solve(inputs: list[list[int]] = MAP) -> int:
    graph = Graph()
    for x, row in enumerate(inputs):
        for y, num in enumerate(row):
            graph.add_node((x, y))
            if x > 0:
                graph.add_edge(from_node=(x - 1, y), to_node=(x, y), distance=num)
            if x < len(inputs):
                graph.add_edge(from_node=(x + 1, y), to_node=(x, y), distance=num)
            if y > 0:
                graph.add_edge(from_node=(x, y - 1), to_node=(x, y), distance=num)
            if y < len(inputs[0]):
                graph.add_edge(from_node=(x, y + 1), to_node=(x, y), distance=num)
    distances = dijkstra(graph=graph, initial=(0, 0))[0]
    return distances[(len(inputs) - 1, len(inputs[0]) - 1)]


Testing using the initial inputs from the AoC site:

In [3]:
def test_min_path():
    inputs = [
        "1163751742",
        "1381373672",
        "2136511328",
        "3694931569",
        "7463417111",
        "1319128137",
        "1359912421",
        "3125421639",
        "1293138521",
        "2311944581",
    ]
    inputs = [list(map(int, x)) for x in inputs]
    min_path_stuff = solve(inputs=inputs)
    assert min_path_stuff == 40, f"{min_path_stuff=}"

test_min_path()

After a lot of fiddling with the solver, I had to ensure that all directions were accounted for in the graph before it would give me the correct solution. Turns out there's likely some backtracking to the left or up somewhere in the map that I assumed wouldn't happen; and I incorrectly set up the graph to go "from" an external node "to" a current one using the incorrect weighting, throwing off the entire formula.

Finally we get to our correct solution which works both in testing and for the larger example:

In [4]:
our_min = solve(inputs=MAP)
print(f"Shortest route distance: {our_min}")

Shortest route distance: 498


From this experience, made respect for the Dijkstra algo. Glad to know I can pull from that to do path finding from now on. 😊

But, it's not over yet:

## Part 2

This one's basically the same, so thankfully we don't have to change the algo. What we do need to change is the map itself.

We'll need to reprocess `MAP` as a template and go through a 5x5 transformation, extending and appending adjusted values. The simplest way to run that calculation for the increased risk levels will involve `x % 9 + 1` to transform position of risk `x` to its next-tier risk level, wrapping around from `9` to `1` as needed.

In [5]:
from copy import deepcopy

new_map = []

for row in MAP:
    new_row = deepcopy(row)
    section = []
    for i in range(4):
        # Append 4 new sections to the new row, adding 1 through 4 to each section
        section.extend([((x + i) % 9) + 1 for x in new_row])
    new_row.extend(section)
    new_map.append(new_row)

for j in range(4):
    start, end = j * 100, (j * 100) + 100
    for row in new_map[start:end]:
        newer_row = row[100:]
        end_section = newer_row[-100:]
        newer_row.extend([(x % 9) + 1 for x in end_section])
        new_map.append(newer_row)


That being done, we just run the same calculation again.

Admittedly, there is likely some more efficient method to do this that I am not at all interested in finding right now. Instead, I dropped this notebook on my desktop PC and let it run overnight.

The calculation to solve the 500x500 map takes about **2 hours** to complete with the algorithm used in this notebook. So, run the following cell again at your own risk!

In [11]:
min_distance_2 = solve(inputs=new_map)
print(min_distance_2)

2901
